In [1]:
import numpy as np
from numpy import linalg as LA
import torch; torch.set_default_dtype(torch.float64)
import torch.nn as nn
import torch.optim as optim
import copy
from copy import deepcopy
import matplotlib.pyplot as plt
# import mymodule as myModules
import matplotlib.cm as cm
from scipy import linalg
from timeit import default_timer as timer
import networkx as nx
import os
import pickle
import datetime
#### Import SLOG packakes
from SLOGmodules import SLOGtools as SLOGtools
from SLOGmodules import SLOGobjective as SLOGobj
from SLOGmodules import SLOGarchitectures as SLOGarchi 
from SLOGmodules import SLOGtraining as SLOGtrainer
from SLOGmodules import SLOGmodel as SLOGmodel
from SLOGmodules import SLOGevaluation as SLOGevaluator
from SLOGmodules import SLOGdata as SLOGdata

#### Import GNN packages
from SLOGmodules import graphTools as graphTools
from SLOGmodules import dataTools as dataTools

from alegnn.modules import architectures as archit
from alegnn.modules import model as model
from alegnn.modules import training as training
from alegnn.modules import evaluation as evaluation
from alegnn.modules import loss as loss
from alegnn.utils.miscTools import writeVarValues
from alegnn.utils.miscTools import saveSeed

Model list

In [2]:
# saveDir_dropbox = '/Users/changye/Dropbox/onlineResults/experiments/'
saveDir_dropbox = r"C:\Users\Chang Ye\Dropbox\onlineResults\experiments"

### ER
nNodes = 20 # Number of nodes
nClasses = 3 # Number of classes (i.e. number of communities)
N_C = 3 # Number of sources per signal
alpha = 1.0

graphType = 'ER' # Type of graph
graphOptions = {} # Dictionary of options to pass to the graphTools.createGraph function
graphOptions['probIntra'] = 0.3 # Probability of drawing edges


modelDirList_0 = ['sourceLocSLOGNET-ER-20220627180032',
                  'sourceLocSLOGNET-ER-20220627180233',
                  'sourceLocSLOGNET-ER-20220627183452',
                  'sourceLocSLOGNET-ER-20220627190900',
                  'sourceLocSLOGNET-ER-20220627194225',
                  'sourceLocSLOGNET-ER-20220627201456',
                  'sourceLocSLOGNET-ER-20220627204704',
                  'sourceLocSLOGNET-ER-20220627211942',
                  'sourceLocSLOGNET-ER-20220627215415',
                  'sourceLocSLOGNET-ER-20220627222703'
                ]
modelDirList_1 = ['sourceLocSLOGNET-ER-20220627233707',
                  'sourceLocSLOGNET-ER-20220628001009',
                  'sourceLocSLOGNET-ER-20220628004336',
                  'sourceLocSLOGNET-ER-20220628011705',
                  'sourceLocSLOGNET-ER-20220628014954',
                  'sourceLocSLOGNET-ER-20220628022340',
                  'sourceLocSLOGNET-ER-20220628025804',
                  'sourceLocSLOGNET-ER-20220628033130',
                  'sourceLocSLOGNET-ER-20220628040621',
                  'sourceLocSLOGNET-ER-20220628044101'
                 ]

modelDirList_2 = ['sourceLocSLOGNET-ER-20220628051822',
                  'sourceLocSLOGNET-ER-20220628055436',
                  'sourceLocSLOGNET-ER-20220628063125',
                  'sourceLocSLOGNET-ER-20220628070942',
                  'sourceLocSLOGNET-ER-20220628074424',
                  'sourceLocSLOGNET-ER-20220628081937',
                  'sourceLocSLOGNET-ER-20220628085328',
                  'sourceLocSLOGNET-ER-20220628092850',
                  'sourceLocSLOGNET-ER-20220628100637',
                  'sourceLocSLOGNET-ER-20220628104252'
                ]


### Random Geometric graph
# nNodes = 20 # Number of nodes
# nClasses = 3 # Number of classes (i.e. number of communities)
# N_C = 2 # Number of sources per signal
# alpha = 1.0

# graphType = 'Random Geometric' # Type of graph
# graphOptions = {} # Dictionary of options to pass to the graphTools.createGraph function
# graphOptions['distance'] = 0.2 # Number of communities

# modelDirList_0 = ['sourceLocSLOGNET-Random Geometric-20220626193044',
#                'sourceLocSLOGNET-Random Geometric-20220626193822',
#                 'sourceLocSLOGNET-Random Geometric-20220626201036',
#                 'sourceLocSLOGNET-Random Geometric-20220626204203',
#                 'sourceLocSLOGNET-Random Geometric-20220626221850',
#                 'sourceLocSLOGNET-Random Geometric-20220626225441',
#                 'sourceLocSLOGNET-Random Geometric-20220626232753',
#                 'sourceLocSLOGNET-Random Geometric-20220626225441',
#                 'sourceLocSLOGNET-Random Geometric-20220626232753',
#                 'sourceLocSLOGNET-Random Geometric-20220627000028'
#                ]

# modelDirList_1 = ['sourceLocSLOGNET-Random Geometric-20220627003548',
#                  'sourceLocSLOGNET-Random Geometric-20220627011123',
#                  'sourceLocSLOGNET-Random Geometric-20220627014519',
#                  'sourceLocSLOGNET-Random Geometric-20220627021813',
#                   'sourceLocSLOGNET-Random Geometric-20220627025343',
#                   'sourceLocSLOGNET-Random Geometric-20220627032731',
#                   'sourceLocSLOGNET-Random Geometric-20220627040029',
#                   'sourceLocSLOGNET-Random Geometric-20220627043400',
#                   'sourceLocSLOGNET-Random Geometric-20220627050606',
#                   'sourceLocSLOGNET-Random Geometric-20220627053850'
#                   ]

# modelDirList_2 = ['sourceLocSLOGNET-Random Geometric-20220627061155',
#                   'sourceLocSLOGNET-Random Geometric-20220627064820',
#                   'sourceLocSLOGNET-Random Geometric-20220627072712',
#                   'sourceLocSLOGNET-Random Geometric-20220627080026',
#                   'sourceLocSLOGNET-Random Geometric-20220627083325',
#                   'sourceLocSLOGNET-Random Geometric-20220627090703',
#                   'sourceLocSLOGNET-Random Geometric-20220627094138',
#                   'sourceLocSLOGNET-Random Geometric-20220627101545',
#                   'sourceLocSLOGNET-Random Geometric-20220627105022',
#                   'sourceLocSLOGNET-Random Geometric-20220627112317'
#                  ]

# print(modelDirList_0)
# for a in modelDirList_0:
#     b = os.path.join(saveDir_dropbox,a)
#     print(b)
    
    

Parameters

In [3]:
### Model parameters
K = 5 # number of layers
C = 1 # constrain constant
filterTrainType = 'g'

### Simulation parameters
simuParas = {}
nTrain_slog = 100000
batchsize_slog = 400
nValid = batchsize_slog
nTest = batchsize_slog
nEpochs = 50

### Data parameters
L = 5
alpha = 1.0

### Graph parameters

# nNodes = 20 # Number of nodes
# nClasses = 3 # Number of classes (i.e. number of communities)
# N_C = 2 # Number of sources per signal

# nNodes = 100 # Number of nodes
# nClasses = 5 # Number of classes (i.e. number of communities)
# N_C = 10 # Number of sources per signal

simuParas['nNodes'] = nNodes
simuParas['nClasses'] = nClasses
simuParas['N_C'] = N_C

# graphType = 'SBM' # Type of graph
# graphOptions = {} # Dictionary of options to pass to the graphTools.createGraph function
# graphOptions['nCommunities'] = nClasses # Number of communities
# graphOptions['probIntra'] = 0.8 # Probability of drawing edges intra communities
# graphOptions['probInter'] = 0.2 # Probability of drawing edges inter communities


# graphType = 'BA' # Type of graph
# graphOptions = {} # Dictionary of options to pass to the graphTools.createGraph function
# graphOptions['alpha'] = 1.0



## Filter type: g or h
# filterType = 'g'
filterType = 'h'

## Signal mode: gaussion or 1
signalMode = 'Gaussion'

## Train mode: Wt or default (Vdiag(\tilde{h})V^T)
trainMode = 'default'
# trainMode = 'Wt'

## Filter mode: Wt or default (Vdiag(\tilde{h})V^T)
filterMode = 'default'
# filterMode = 'Wt'

## Selection mode: random or nodes with top-N_C degree
selectMode = 'random'

## Noise level
noiseLevel = 0.0

simuParas['nTrain_slog'] = nTrain_slog
simuParas['batchsize_slog'] = batchsize_slog
simuParas['nValid'] = nValid
simuParas['nTest'] = nTest
simuParas['L'] = L
simuParas['noiseLevel'] = noiseLevel
simuParas['filterType'] = filterType
simuParas['signalMode'] = signalMode
simuParas['trainMode'] = trainMode
simuParas['filterMode'] = filterMode
simuParas['selectMode'] = selectMode
simuParas['graphType'] = graphType 
simuParas['alpha'] = alpha
simuParas['nEpochs'] = nEpochs


tMax = None # Maximum number of diffusion times (W^t for t < tMax)
tMax = 3
simuParas['tMax'] = tMax

## model parameters
modelParas = {}
modelParas['filterTrainType'] = filterTrainType
modelParas['C'] = C
modelParas['K'] = K

## Experiment parameters
expParas = {}
expParas['nRealiz'] = 10

In [4]:

# dataTest = test_sample_generate(nNodes, S, P, nTest, gso, L = L, noiseLevel = noiseLevel, alpha = alpha, filterType = filterType)


In [5]:
# print(myModel.archit.state_dict())

SLOG-Net Model

In [6]:
C = 1
K = 5
filterTrainType = 'g'
# filterTrainType = 'h'
# tMax = 5
thisLoss = SLOGtools.myLoss
thisEvaluator = SLOGevaluator.evaluate

thisObject = SLOGobj.myFunction_slog_1

# thisObject = SLOGobj.myFunction_slog_2
# SLOG_net = SLOGarchi.GraphSLoG_v2(V_slog,nNodes,C,K, thisObject)

import torch.optim as optim

model_name = 'SLOG-Net'
device = 'gpu'
optimAlg = 'ADAM'
learningRate = 0.001
beta1 = 0.9
beta2 = 0.999

N_test = 10
N_model = len(modelDirList_0)
result_test_1 = np.zeros([N_model,N_test,2])

n_model = 0
for modelDir in modelDirList_0:
    label = 'Best'
    saveDir = os.path.join(saveDir_dropbox,modelDir)
    print(saveDir)
    gsoName = 'gso-' + graphType + '.npy'
    print(gsoName)
    gsoDir = os.path.join(saveDir, gsoName)
    print(gsoDir)
    gso = np.load(gsoDir)
    G = SLOGtools.Graph(graphType, nNodes, graphOptions)
    G.setGSO(gso)
    
    sourceNodes, communityLabels,communityList = SLOGtools.computeSourceNodes_slog(gso, nClasses, N_C, mode = 'random')
    d_slog,An_slog, eigenvalues_slog, V_slog   = SLOGtools.get_eig_normalized_adj(gso)
    SLOG_net = SLOGarchi.GraphSLoG_v1(V_slog,nNodes,C,K, thisObject)
    thisOptim = optim.Adam(SLOG_net.parameters(), lr = learningRate, betas = (beta1,beta2))
    thisTrainer = SLOGtrainer.slog_Trainer    
    loadedModel = SLOGmodel.Model(SLOG_net,thisLoss,thisOptim, thisTrainer,thisEvaluator,device, model_name,  saveDir, saveDir_dropbox = saveDir_dropbox )
    loadedModel.load_from_dropBox(saveDir, label = label)    

    for n_test in range(N_test):
        data = SLOGdata.SLOG_ClassificationData(G, nTest, nTest, nTest, sourceNodes, communityList, communityLabels, V_slog, eigenvalues_slog, L = L, tMax = tMax, alpha = alpha, filterMode = filterMode, selectMode = selectMode, signalMode = signalMode, filterType = filterType, noiseLevel = noiseLevel)
        Y_test,X_test,targets,g_test = data.get_test_Samples()
        g_test = torch.tensor(g_test)
        x_test = np.reshape(X_test.T, nNodes*nTest)
        x_hat, g_hat = loadedModel.archit(Y_test)
        Z = torch.tensor(linalg.khatri_rao(np.dot(np.transpose(Y_test),V_slog),V_slog),requires_grad=False)
        g_hat = g_hat.clone().detach().requires_grad_(False)
        g_hat = nNodes*g_hat/sum(g_hat)        
        x_hat = torch.matmul(Z,g_hat)        
#         x_hat = x_hat.clone().detach().requires_grad_(False)
        re_x_1 = LA.norm(x_hat - x_test)/LA.norm(x_test)
        re_g_1 = LA.norm(g_test - g_hat)/LA.norm(g_test)
        re_x_2 = LA.norm(x_hat + x_test)/LA.norm(x_test)
        re_g_2 = LA.norm(g_test + g_hat)/LA.norm(g_test)        
        if re_g_1 > re_g_2:
            re_g = re_g_2
            re_x = re_x_2
        else:
            re_g = re_g_1
            re_x = re_x_1            
        print(re_x,re_g)
        result_test_1[n_model,n_test,0] = re_x
        result_test_1[n_model,n_test,1] = re_g
    n_model += 1

        
    
    print(modelDir)
#     print(loadedModel)
# result_train = myModel.train(data,nEpochs, batchsize_slog, validationInterval = 40,trainMode = trainMode,tMax = tMax, filterTrainType = filterTrainType) # model, data, nEpochs, batchSize


C:\Users\Chang Ye\Dropbox\onlineResults\experiments\sourceLocSLOGNET-ER-20220627180032
gso-ER.npy
C:\Users\Chang Ye\Dropbox\onlineResults\experiments\sourceLocSLOGNET-ER-20220627180032\gso-ER.npy
[[0.         0.36868436 0.75292292 0.8311683  0.55541641 0.67240016
  0.17466388 0.06766235 0.39800499 0.52304948 0.70746061 0.05716466
  0.69123648 0.08829638 0.34129701 0.7663396  0.50462964 0.44969306
  0.034298   0.32808291]
 [0.36868436 0.         0.36530957 0.20535009 0.16371018 0.04025081
  0.27582418 0.28166184 0.53913586 0.36391512 0.66846836 0.08607922
  0.825832   0.35961388 0.99242483 0.04533561 0.13215548 0.51932293
  0.94760658 0.70697946]
 [0.75292292 0.36530957 0.         0.57127685 0.55663426 0.68214908
  0.160818   0.99628193 0.78168033 0.47480941 0.82173875 0.71415976
  0.46000998 0.83184163 0.60618141 0.47541877 0.1013043  0.36693197
  0.2161905  0.19491962]
 [0.8311683  0.20535009 0.57127685 0.         0.26072052 0.91274175
  0.97741691 0.46712081 0.30658542 0.82795684 0.2

C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:489: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  communityNode_set = np.array(communityList)
C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:554: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  signals = torch.tensor(x_slog + noiseLevel*np.random.normal(0, 1, [nTotal,G.N]))# nTotal x N


Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.851782687948732 1.0955729463172932
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[0.36984778 0.58583954 0.64202542 0.6780768  0.68646978 0.74663675
 0.77599143 0.80780729 1.682337   1.60557739 1.49799703 1.39306299
 0.88614978 0.93501025 0.95682653 1.26286768 1.20620991 1.03771271
 1.11041718 1.13313678]
Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
1.3223410304727075 1.0638056097198252
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[ 0.2169062  32.15934778 -1.80536874 -1.17358256 -1.09411694 -0.77813517
 -0.70139301 -0.64393421 -0.52867398 -0.52015476 -0.50977876 -0.50173271


C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:489: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  communityNode_set = np.array(communityList)
C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:554: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  signals = torch.tensor(x_slog + noiseLevel*np.random.normal(0, 1, [nTotal,G.N]))# nTotal x N


Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.10040305647782843 0.10152457392485115
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[ -0.21694297 -11.49011405  18.67879058   0.60931344   0.60242506
   0.59938232   1.77075982   1.27230241   0.59479643   1.01173246
   0.8314664    0.7502423    0.59328047   0.59569784   0.60627759
   0.60333952   0.6303206    0.66325603   0.64180217   0.65187158]
Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.205110911437428 0.14507743870073267
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[0.54844633 0.73502807 0.74274281 1.93079734 1.58333902 1.45546746
 0.77308115 0.78807239 1.26267411 0.

...

Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.10562083940128557 0.08349776959058633
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[-0.5575911   2.31992491  2.51342276  1.88645816  1.28515583  1.22352536
  1.10009578  0.97752801  0.88761761  0.8569297   0.85996087  0.77659621
  0.74374984  0.72073067  0.71536828  0.71496245  0.72229504  0.75594005
  0.75118228  0.74614726]
Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.06594292207293022 0.06951834034377366
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[0.26839734 0.3613644  4.79577619 2.8559273  0.39305556 1.50629976
 1.30203051 1.1072393  0.44491191 0.45385045 0.9215929

C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:489: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  communityNode_set = np.array(communityList)
C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:554: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  signals = torch.tensor(x_slog + noiseLevel*np.random.normal(0, 1, [nTotal,G.N]))# nTotal x N


Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
3.430781413255821 1.9662300016359198
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[0.50218574 1.8352471  0.79032086 1.42162089 1.32249846 1.25176892
 0.81387727 0.82849109 0.83484799 0.84364476 1.11435424 1.0625021
 1.03124704 0.97901874 0.96335295 0.91499175 0.88713697 0.88501063
 0.85438501 0.86349749]
Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.0808355196514672 0.08018262938883676
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[0.36313721 1.33245977 0.6683489  1.27989671 1.26148793 1.24583767
 0.72776408 0.77595456 0.80061135 0.83854937 1.20538852 1.1841126
 1.16846809 1

C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:489: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  communityNode_set = np.array(communityList)
C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:554: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  signals = torch.tensor(x_slog + noiseLevel*np.random.normal(0, 1, [nTotal,G.N]))# nTotal x N


Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.02679074815121059 0.024299295697457285
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[0.38964145 0.70027044 0.76156266 0.80100468 0.8660562  0.98826522
 0.94365588 1.05793625 1.08219777 1.11338625 0.99266186 1.04744179
 1.09082523 1.1637711  1.12878551 1.14371337 1.17529119 1.18409051
 1.1840722  1.18537043]
Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.1350285246577992 0.24562469732873266
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[  1.06370076   9.84311639  81.37413029 -25.49107038  -8.59113743
  -1.89388529  -5.0206093   -1.86089137  -1.85907387  -1.86636326
  -4.0309

In [7]:
print(result_test_1)

[[[0.85178269 1.09557295]
  [1.32234103 1.06380561]
  [0.50833253 0.55778749]
  [0.69359877 0.67381398]
  [0.52525874 0.49421196]
  [1.20437419 1.26727056]
  [1.48939107 1.29600478]
  [0.69185596 0.69574071]
  [0.43389164 0.4442046 ]
  [0.5698644  0.85438805]]

 [[0.10040306 0.10152457]
  [0.20511091 0.14507744]
  [0.85794686 1.57268965]
  [0.13510341 0.19221714]
  [0.10049399 0.09198588]
  [0.1156473  0.10761809]
  [0.09391911 0.10728841]
  [0.08490107 0.09389732]
  [0.69297773 0.60047184]
  [0.06212109 0.0614291 ]]

 [[0.03841828 0.02063903]
  [0.02891212 0.01909774]
  [0.02656268 0.02007901]
  [0.04602961 0.03363358]
  [0.02292005 0.01507215]
  [0.10682099 0.17708595]
  [0.02034874 0.02558484]
  [1.5662465  1.98254694]
  [0.01481676 0.01508079]
  [0.01828882 0.01841566]]

 [[0.01491141 0.01374549]
  [0.01502915 0.01392826]
  [0.02014797 0.02090908]
  [0.02585725 0.01986323]
  [0.02143987 0.01942495]
  [0.97985157 0.17885347]
  [0.04110451 0.04411694]
  [0.03082791 0.04683287]
  [0.0

In [8]:
C = 1
K = 5
filterTrainType = 'g'
# filterTrainType = 'h'
# tMax = 5
thisLoss = SLOGtools.myLoss
thisEvaluator = SLOGevaluator.evaluate

thisObject = SLOGobj.myFunction_slog_1

# thisObject = SLOGobj.myFunction_slog_2
# SLOG_net = SLOGarchi.GraphSLoG_v2(V_slog,nNodes,C,K, thisObject)

import torch.optim as optim

model_name = 'SLOG-Net'
device = 'gpu'
optimAlg = 'ADAM'
learningRate = 0.001
beta1 = 0.9
beta2 = 0.999

noiseLevel = 0.01
N_test = 10
N_model = len(modelDirList_1)
result_test_2 = np.zeros([N_model,N_test,2])

n_model = 0
for modelDir in modelDirList_1:
    label = 'Best'
    saveDir = os.path.join(saveDir_dropbox,modelDir)
    print(saveDir)
    gsoName = 'gso-' + graphType + '.npy'
    print(gsoName)
    gsoDir = os.path.join(saveDir, gsoName)
    print(gsoDir)
    gso = np.load(gsoDir)
    G = SLOGtools.Graph(graphType, nNodes, graphOptions)
    G.setGSO(gso)
    
    sourceNodes, communityLabels,communityList = SLOGtools.computeSourceNodes_slog(gso, nClasses, N_C, mode = 'random')
    d_slog,An_slog, eigenvalues_slog, V_slog   = SLOGtools.get_eig_normalized_adj(gso)
    SLOG_net = SLOGarchi.GraphSLoG_v1(V_slog,nNodes,C,K, thisObject)
    thisOptim = optim.Adam(SLOG_net.parameters(), lr = learningRate, betas = (beta1,beta2))
    thisTrainer = SLOGtrainer.slog_Trainer    
    loadedModel = SLOGmodel.Model(SLOG_net,thisLoss,thisOptim, thisTrainer,thisEvaluator,device, model_name,  saveDir, saveDir_dropbox = saveDir_dropbox )
    loadedModel.load_from_dropBox(saveDir, label = label)    

    for n_test in range(N_test):
        data = SLOGdata.SLOG_ClassificationData(G, nTest, nTest, nTest, sourceNodes, communityList, communityLabels, V_slog, eigenvalues_slog, L = L, tMax = tMax, alpha = alpha, filterMode = filterMode, selectMode = selectMode, signalMode = signalMode, filterType = filterType, noiseLevel = noiseLevel)
        Y_test,X_test,targets,g_test = data.get_test_Samples()
        g_test = torch.tensor(g_test)
        x_test = np.reshape(X_test.T, nNodes*nTest)
        x_hat, g_hat = loadedModel.archit(Y_test)
        Z = torch.tensor(linalg.khatri_rao(np.dot(np.transpose(Y_test),V_slog),V_slog),requires_grad=False)
        g_hat = g_hat.clone().detach().requires_grad_(False)
        g_hat = nNodes*g_hat/sum(g_hat)        
        x_hat = torch.matmul(Z,g_hat)        
#         x_hat = x_hat.clone().detach().requires_grad_(False)
        re_x_1 = LA.norm(x_hat - x_test)/LA.norm(x_test)
        re_g_1 = LA.norm(g_test - g_hat)/LA.norm(g_test)
        re_x_2 = LA.norm(x_hat + x_test)/LA.norm(x_test)
        re_g_2 = LA.norm(g_test + g_hat)/LA.norm(g_test)        
        if re_g_1 > re_g_2:
            re_g = re_g_2
            re_x = re_x_2
        else:
            re_g = re_g_1
            re_x = re_x_1            
        print(re_x,re_g)
        result_test_2[n_model,n_test,0] = re_x
        result_test_2[n_model,n_test,1] = re_g
    n_model += 1

        
    
    print(modelDir)
#     print(loadedModel)
# result_train = myModel.train(data,nEpochs, batchsize_slog, validationInterval = 40,trainMode = trainMode,tMax = tMax, filterTrainType = filterTrainType) # model, data, nEpochs, batchSize


C:\Users\Chang Ye\Dropbox\onlineResults\experiments\sourceLocSLOGNET-ER-20220627233707
gso-ER.npy
C:\Users\Chang Ye\Dropbox\onlineResults\experiments\sourceLocSLOGNET-ER-20220627233707\gso-ER.npy
[[0.         0.61798553 0.04921206 0.97927124 0.15781636 0.5899639
  0.02079438 0.7885741  0.1917952  0.51538806 0.12826356 0.29175055
  0.97100529 0.22171369 0.71687004 0.46616175 0.32483775 0.54229404
  0.46209136 0.15449874]
 [0.61798553 0.         0.31226917 0.58434757 0.59623829 0.20697023
  0.44635093 0.7709018  0.28699292 0.07266621 0.49092098 0.73704891
  0.33851233 0.87059998 0.89984992 0.21012677 0.30039161 0.30721701
  0.93814409 0.62201985]
 [0.04921206 0.31226917 0.         0.81996052 0.53811421 0.41113352
  0.71237998 0.37319336 0.27306466 0.38446077 0.3496972  0.97256889
  0.21428993 0.3566679  0.78200716 0.63542495 0.46810894 0.11535361
  0.53955487 0.41612568]
 [0.97927124 0.58434757 0.81996052 0.         0.50562506 0.80443911
  0.93871475 0.0866005  0.09760073 0.53056456 0.39

C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:489: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  communityNode_set = np.array(communityList)
C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:554: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  signals = torch.tensor(x_slog + noiseLevel*np.random.normal(0, 1, [nTotal,G.N]))# nTotal x N


Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.09349576115320779 0.09055170359129358
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[-0.02182102 -0.0900744   0.02963235  0.02964636 19.17492523  0.19686167
  0.14451088  0.08983499  0.06679044  0.04884883  0.04582087  0.03701503
  0.03601272  0.02952081  0.0295992   0.02985565  0.03139399  0.03108588
  0.03043321  0.03010732]
Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.07329574835832084 0.06499164373935885
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[-2.21390439  1.27501387  1.24520067  1.24577132  1.1263242   1.09696151
  1.09030138  1.08228159  1.08314254  1.09764604

C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:489: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  communityNode_set = np.array(communityList)
C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:554: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  signals = torch.tensor(x_slog + noiseLevel*np.random.normal(0, 1, [nTotal,G.N]))# nTotal x N


Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.12989954558039432 0.2300819833295977
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[ 0.06145813 -0.09615588 -0.09812273 -0.10295213 -0.11270903 -0.12050737
 -0.12250186 -0.13457818 -0.12800429 -0.16193355  1.16655809 24.54676662
 -1.21608479 -1.71018294 -0.19147972 -0.21656615 -0.46220893 -0.3435798
 -0.26789066 -0.28932485]
Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.9143976780380687 1.1399767409672228
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[-6.46239615  1.25307791  1.15554957  1.07187319  1.02511842  1.02044023
  1.02134516  1.03682679  1.02665994  1.09866271  2.

...

Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.9656459667218779 0.5447671315981146
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[0.98304731 1.15943462 1.12443156 1.04367869 1.07898873 1.04226178
 1.01996556 1.00822767 0.99720464 0.98076656 0.98170471 0.96682148
 0.96747212 0.95946792 0.95040809 0.94624209 0.94365927 0.95262523
 0.9463707  0.94722128]
Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.2452463568121179 0.14457493927217996
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[1.20152992 0.84867653 0.91405532 0.96161293 0.98496047 1.0291167
 0.9475796  1.05583221 0.94495355 0.94870997 1.06290307 0.95674238
 1.05905852

C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:489: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  communityNode_set = np.array(communityList)
C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:554: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  signals = torch.tensor(x_slog + noiseLevel*np.random.normal(0, 1, [nTotal,G.N]))# nTotal x N


Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.178451634681467 0.20125285572333507
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[ -2.05243696 -15.91741883 -24.95000384  18.95098184   1.75266172
   1.77539431   1.7973293    1.81870982   1.90866763   1.9569303
   5.46919422   4.67285987   4.13629412   3.80363473   2.11839926
   2.29142553   2.35160591   2.55030242   2.65483032   2.91063832]
Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.13964481638374526 0.10022662582989403
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[0.33801182 0.73757403 0.76890128 0.90056193 1.04008589 1.0216941
 1.02052291 1.02204898 1.03675169 1.04

C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:489: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  communityNode_set = np.array(communityList)
C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:554: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  signals = torch.tensor(x_slog + noiseLevel*np.random.normal(0, 1, [nTotal,G.N]))# nTotal x N


Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.08989072595400621 0.08423547157644862
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[-0.50100547  0.53290997  0.54790377  0.56590006  0.63435053  0.6631849
  0.67987579  0.71731325  0.73585967  3.76986155  2.17570416  0.81246576
  0.83762673  0.86815886  0.89669269  0.95120668  1.49468347  1.31977045
  1.20611462  1.09142257]
Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.11550433474743958 0.10860508649689019
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[1.52273823 1.87222375 1.65083557 1.45689151 1.05493895 0.96207485
 0.91940687 0.84538818 0.81771757 0.96358336 0.91106597

In [9]:
print(result_test_2)

[[[0.09349576 0.0905517 ]
  [0.07329575 0.06499164]
  [0.13858452 0.08143544]
  [0.15928491 0.23048614]
  [0.07793868 0.05753458]
  [0.07090838 0.05210186]
  [0.99799273 0.99999428]
  [0.9942352  0.18594153]
  [0.12098254 0.08983092]
  [0.11328145 0.07632039]]

 [[0.12989955 0.23008198]
  [0.91439768 1.13997674]
  [0.32103117 0.39017213]
  [0.95909546 1.00218957]
  [0.98874686 0.11150894]
  [0.06114534 0.05638698]
  [1.1499321  0.68355624]
  [0.06935521 0.07165332]
  [0.07102181 0.05899549]
  [0.06191424 0.05023231]]

 [[0.10420686 0.16468395]
  [0.08130744 0.07294419]
  [0.1066536  0.18102426]
  [0.07343361 0.06526198]
  [0.23224736 0.2284517 ]
  [0.0786762  0.06359836]
  [1.00240842 0.80591062]
  [0.07779526 0.06838479]
  [0.11155435 0.07975566]
  [0.25905316 0.22882036]]

 [[0.94975473 0.38583531]
  [0.05801262 0.06884251]
  [0.04981442 0.03970101]
  [0.09269942 0.10843531]
  [0.05514883 0.05132533]
  [0.0451191  0.03346686]
  [0.14819263 0.25617349]
  [0.03841619 0.02705556]
  [0.0

In [10]:
C = 1
K = 5
filterTrainType = 'g'
# filterTrainType = 'h'
# tMax = 5
thisLoss = SLOGtools.myLoss
thisEvaluator = SLOGevaluator.evaluate

thisObject = SLOGobj.myFunction_slog_1

# thisObject = SLOGobj.myFunction_slog_2
# SLOG_net = SLOGarchi.GraphSLoG_v2(V_slog,nNodes,C,K, thisObject)

import torch.optim as optim

model_name = 'SLOG-Net'
device = 'gpu'
optimAlg = 'ADAM'
learningRate = 0.001
beta1 = 0.9
beta2 = 0.999

noiseLevel = 0.05
N_test = 10
N_model = len(modelDirList_2)
result_test_3 = np.zeros([N_model,N_test,2])

n_model = 0
for modelDir in modelDirList_2:
    label = 'Best'
    saveDir = os.path.join(saveDir_dropbox,modelDir)
    print(saveDir)
    gsoName = 'gso-' + graphType + '.npy'
    print(gsoName)
    gsoDir = os.path.join(saveDir, gsoName)
    print(gsoDir)
    gso = np.load(gsoDir)
    G = SLOGtools.Graph(graphType, nNodes, graphOptions)
    G.setGSO(gso)
    
    sourceNodes, communityLabels,communityList = SLOGtools.computeSourceNodes_slog(gso, nClasses, N_C, mode = 'random')
    d_slog,An_slog, eigenvalues_slog, V_slog   = SLOGtools.get_eig_normalized_adj(gso)
    SLOG_net = SLOGarchi.GraphSLoG_v1(V_slog,nNodes,C,K, thisObject)
    thisOptim = optim.Adam(SLOG_net.parameters(), lr = learningRate, betas = (beta1,beta2))
    thisTrainer = SLOGtrainer.slog_Trainer    
    loadedModel = SLOGmodel.Model(SLOG_net,thisLoss,thisOptim, thisTrainer,thisEvaluator,device, model_name,  saveDir, saveDir_dropbox = saveDir_dropbox )
    loadedModel.load_from_dropBox(saveDir, label = label)    

    for n_test in range(N_test):
        data = SLOGdata.SLOG_ClassificationData(G, nTest, nTest, nTest, sourceNodes, communityList, communityLabels, V_slog, eigenvalues_slog, L = L, tMax = tMax, alpha = alpha, filterMode = filterMode, selectMode = selectMode, signalMode = signalMode, filterType = filterType, noiseLevel = noiseLevel)
        Y_test,X_test,targets,g_test = data.get_test_Samples()
        g_test = torch.tensor(g_test)
        x_test = np.reshape(X_test.T, nNodes*nTest)
        x_hat, g_hat = loadedModel.archit(Y_test)
        Z = torch.tensor(linalg.khatri_rao(np.dot(np.transpose(Y_test),V_slog),V_slog),requires_grad=False)
        g_hat = g_hat.clone().detach().requires_grad_(False)
        g_hat = nNodes*g_hat/sum(g_hat)        
        x_hat = torch.matmul(Z,g_hat)        
#         x_hat = x_hat.clone().detach().requires_grad_(False)
        
        re_x_1 = LA.norm(x_hat - x_test)/LA.norm(x_test)
        re_g_1 = LA.norm(g_test - g_hat)/LA.norm(g_test)
        re_x_2 = LA.norm(x_hat + x_test)/LA.norm(x_test)
        re_g_2 = LA.norm(g_test + g_hat)/LA.norm(g_test)        
        if re_g_1 > re_g_2:
            re_g = re_g_2
            re_x = re_x_2
        else:
            re_g = re_g_1
            re_x = re_x_1            
        print(re_x,re_g)
        result_test_3[n_model,n_test,0] = re_x
        result_test_3[n_model,n_test,1] = re_g
    n_model += 1

        
    
    print(modelDir)
#     print(loadedModel)
# result_train = myModel.train(data,nEpochs, batchsize_slog, validationInterval = 40,trainMode = trainMode,tMax = tMax, filterTrainType = filterTrainType) # model, data, nEpochs, batchSize


C:\Users\Chang Ye\Dropbox\onlineResults\experiments\sourceLocSLOGNET-ER-20220628051822
gso-ER.npy
C:\Users\Chang Ye\Dropbox\onlineResults\experiments\sourceLocSLOGNET-ER-20220628051822\gso-ER.npy
[[0.         0.42086904 0.71153816 0.39703335 0.37178412 0.33500006
  0.07064016 0.68269978 0.14206399 0.50557527 0.25626378 0.59915552
  0.60783668 0.4309617  0.11571906 0.11066465 0.17803864 0.36307254
  0.08791382 0.52708934]
 [0.42086904 0.         0.89956084 0.11192011 0.88071674 0.67926828
  0.29393723 0.99573922 0.43907334 0.58398637 0.31673211 0.95432787
  0.26327328 0.41720209 0.66722068 0.23049702 0.76072107 0.40292407
  0.00570238 0.49060424]
 [0.71153816 0.89956084 0.         0.25414689 0.47431005 0.58131564
  0.75084709 0.99627701 0.92220804 0.15878954 0.390284   0.9350174
  0.29901733 0.72273919 0.48440485 0.94276806 0.84328228 0.98195124
  0.19426133 0.25382254]
 [0.39703335 0.11192011 0.25414689 0.         0.60540508 0.32108638
  0.75698437 0.91620414 0.18293051 0.98580371 0.72

C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:489: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  communityNode_set = np.array(communityList)
C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:554: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  signals = torch.tensor(x_slog + noiseLevel*np.random.normal(0, 1, [nTotal,G.N]))# nTotal x N


Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.1305567805853239 0.03759050514176136
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[ 0.03792403 -0.06288791 21.64402215 -0.06248597 -0.24187457 -0.20742852
 -0.13965714 -0.06262989 -0.09983421 -0.06387508 -0.06515026 -0.0849716
 -0.08449279 -0.07975744 -0.07787082 -0.07222608 -0.07360506 -0.06889119
 -0.0669534  -0.06735426]
Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.14202689929791928 0.046841991590492765
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[1.62844726 1.07186636 1.32715868 1.02986907 1.16925259 1.14245686
 1.05133507 0.9761998  0.94611926 0.91856121 0.89268194

C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:489: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  communityNode_set = np.array(communityList)
C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:554: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  signals = torch.tensor(x_slog + noiseLevel*np.random.normal(0, 1, [nTotal,G.N]))# nTotal x N


Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.1551217773104908 0.08139618867711375
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[1.08747194 0.53590283 0.51741015 0.51761768 0.52442791 0.53703272
 0.55532753 0.57644871 0.60596459 0.64566375 0.66399867 2.73076863
 2.24320579 1.82366874 1.48550172 1.31422865 0.79789142 0.85214141
 1.01143913 0.97388802]
Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.13146936541386522 0.022362197538535546
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[1.86192109 1.08454714 0.99776046 0.92419518 0.8968472  0.87123679
 0.85051982 0.83652561 0.82576399 0.82002723 0.81948937 1.36146198
 1.2451

...

Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.15293492492135308 0.04601396354588065
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[ 0.66758399 32.01841215 -2.15585443 -1.55338431 -0.34520258 -1.12358131
 -1.00921392 -0.85100604 -0.35309687 -0.68837372 -0.63060168 -0.59651334
 -0.54550447 -0.36484978 -0.3746422  -0.45463512 -0.38914415 -0.39543498
 -0.43331993 -0.4216373 ]
Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.1853479427473426 0.09279534150297648
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[0.27181237 0.37757022 0.4368836  0.46292638 2.95634538 0.50357213
 0.52220053 0.55983563 2.20340525 0.6277774  0.66666717

C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:489: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  communityNode_set = np.array(communityList)
C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:554: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  signals = torch.tensor(x_slog + noiseLevel*np.random.normal(0, 1, [nTotal,G.N]))# nTotal x N


Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.13326317744447505 0.02704156035519086
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[0.46175226 0.66706973 1.63482168 0.74402939 0.76880603 0.78994044
 0.81268663 0.85355577 0.9005944  0.91622812 0.96817493 1.33249956
 1.3219097  1.2384133  1.1928386  1.02324077 1.04330489 1.14059659
 1.10894349 1.08059372]
Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.5637118477551648 1.052471186019271
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[3.77160195 1.28526505 0.66007512 1.12661514 1.08559103 1.05302977
 1.01999044 0.96474308 0.9063437  0.88803299 0.83135761 0.66507752
 0.6660563

C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:489: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  communityNode_set = np.array(communityList)
C:\Users\Chang Ye\python\github\slog-net\slog-net-master\graph-neural-networks-master\SLOGmodules\SLOGdata.py:554: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  signals = torch.tensor(x_slog + noiseLevel*np.random.normal(0, 1, [nTotal,G.N]))# nTotal x N


Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
0.16605160260113136 0.09374525757513545
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[0.4870952  1.11047294 1.09929164 0.94194177 1.08345841 1.06714941
 1.05696979 0.98266607 0.99448367 0.99713018 1.02987346 1.02551593
 1.02189505 1.01853663 1.01055553 1.01538656 1.01507914 1.01454574
 1.01395723 1.01399564]
Signal shape: torch.Size([1200, 20])
key: train
key: valid
key: test
torch.Size([400, 20])
Sample shape Y: torch.Size([20, 400]) , X: torch.Size([20, 400]) , targets: (400,)
1.0857654278307194 0.5371535066935406
<class 'numpy.float64'>
Number of input nodes per classes: 3 ; number of classes 3
Generating h filter
[1.84251522 0.61450659 0.64168039 1.10754004 0.68254492 0.7301281
 0.76432015 1.11510595 1.11814114 1.11870638 0.89049532 0.92125416
 0.9525812

In [11]:
print(result_test_3)

[[[0.13055678 0.03759051]
  [0.1420269  0.04684199]
  [0.15012005 0.05835918]
  [0.14552975 0.04776328]
  [0.34783863 0.74688516]
  [0.14781987 0.0494428 ]
  [0.16826447 0.09086431]
  [0.14148359 0.04094321]
  [0.12935651 0.03072721]
  [0.1642289  0.06801281]]

 [[0.15512178 0.08139619]
  [0.13146937 0.0223622 ]
  [0.16896827 0.08045469]
  [0.13138664 0.02290058]
  [0.16352569 0.11599932]
  [0.13778181 0.02564538]
  [0.13772569 0.02965766]
  [1.43939408 0.59029023]
  [0.13882206 0.03206424]
  [0.12865139 0.02550382]]

 [[0.21564026 0.34129425]
  [0.74127988 0.84428258]
  [0.6123866  0.33774445]
  [0.21394511 0.09210345]
  [1.13424198 0.97845322]
  [0.19557645 0.09134487]
  [0.35968928 0.43671489]
  [0.17233269 0.08440049]
  [0.60532718 0.62095501]
  [0.16707317 0.07855594]]

 [[0.30175491 0.60771439]
  [0.19211758 0.06197804]
  [1.06318571 0.5059224 ]
  [0.1529991  0.07643043]
  [0.19344923 0.2038123 ]
  [0.15630429 0.08795566]
  [0.17094653 0.08658665]
  [0.61707696 0.61078967]
  [0.1